In [6]:
#Importações
import pandas as pd
from sklearn.model_selection import train_test_split
import mlflow
import pycaret.classification as pc

import matplotlib.pyplot as plt
from sklearn.model_selection import validation_curve

import numpy as np
from sklearn.metrics import make_scorer, f1_score
from sklearn.metrics import log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier


In [7]:
# Para usar o sqlite como repositorio
mlflow.set_tracking_uri("sqlite:///mlflow.db")

experiment_name = 'Projeto Kobe '
experiment = mlflow.get_experiment_by_name(experiment_name)
if experiment is None:
    experiment_id = mlflow.create_experiment(experiment_name)
    experiment = mlflow.get_experiment(experiment_id)
experiment_id = experiment.experiment_id

Preparação dos dados

In [9]:

#MLflow
with mlflow.start_run(experiment_id=experiment_id, run_name='PreparacaoDados'):
   
    percentual_treino = 0.8
    
  
    df_dev = pd.read_parquet("../data/raw/dataset_kobe_dev.parquet")
    
    # Colunas selecionadas
    colunas_selecionadas = ['lat','lon','minutes_remaining', 'period', 'playoffs', 'shot_distance', 'shot_made_flag']

    # Carregar o DataFrame a partir do arquivo parquet
    df_dev = pd.read_parquet("../data/raw/dataset_kobe_dev.parquet")

    # Remover todas as colunas que não estão na lista 'colunas_selecionadas'
    colunas_para_remover = [coluna for coluna in df_dev.columns if coluna not in colunas_selecionadas]
    df_dev.drop(columns=colunas_para_remover, inplace=True)
    df_dev = df_dev.dropna()

    df_dev.to_parquet('../data/processed/data_filtered.parquet', index=False)
    df_dev['shot_made_flag'] = df_dev['shot_made_flag'].astype(int)
    xtrain, xtest, ytrain, ytest =  train_test_split(df_dev[['lat', 'lon', 'minutes_remaining', 'period', 'playoffs', 'shot_distance']],
                                                     df_dev['shot_made_flag'],
                                                     test_size= percentual_treino,
                                                     stratify=df_dev['shot_made_flag']
                                                   )
   
    xtrain['shot_made_flag'] = ytrain
    ytrain['shot_made_flag'] = ytest
   
    xtrain.to_parquet("../data/processed/base_train.parque")
    xtest.to_parquet("../data/processed/base_test.parque")   
   
     # Registre os parâmetros e métricas no MLflow
    mlflow.log_param('percentual_test', 1 - percentual_treino)
    mlflow.log_param('colunas_selecionadas', colunas_selecionadas)
    mlflow.log_metric('qtd_linhas_treino', xtrain.shape[0])
    mlflow.log_metric('qtd_linhas_test', xtest.shape[0])